In [ ]:
robot extract --method MIREOT --input cpc21.owl --lower-term GO:CPC21Code49 --output cpc-transport.owl

## Filter ICF maps into groups for experiments as bridging axioms

As a programmer, write the following python code without executing it. 


topdir = '/Users/tu/workspace/harmonization'   
mapdir = F"{topdir}/maps/anatomy"   
ontdir = F"{topdir}/ontology/renamedIRIs"   

Define isLeafNode(iri) as as Boolean function that's true iff len(iri.descendants()) == 1  

using the owlready2 API, read an OWL file F"{ontdir}/whofic-renamed.owl" as ont,   
Read the mapping table F"{mapdir}/ICF-ICD-anatomy-v1.xlsx" Excel spreadsheet.   
Use the first row as the header row.   
1. Define 6 new dataframes DF1, DF2, DF3, DF4, DF5, DF6   
2. Define SurfaceTopIRI as onto.search_one(iri='http://id.who.int/icd/entity/687250607')   
3. For each row after the header row,   
      subjectIRI = onto.search_one(iri='http://id.who.int/icf/entity/' + (subject_id as a string)) of the row   
      objectIRI = onto.search_one(iri='http://id.who.int/icd/entity/' + (object_id as a string)) of the row   
4. If objectIRI is a descendant of SurfaceTopIRI, then     
   add the row to DF1 using the panda.concat function    
   else add the row to DF2 using the panda.concat function   
5. if isLeafNode(subjectIRI) is true   
    add the row to DF3 using the panda.concat function   
  else add the row to DF4   
6. if (the objectIRI is a descendant of SurfaceTopIRI and not isLeafNode(subjectIRI) )   
    add the row to DF5   
  else add the row to DF6   
7. Write each of (DF1, DF2, DF3, DF4, DF5, DF6) as excel files   
  (TargetSurfaceTopography.xlsx, TargetNotSurfaceTopography.xlsx, SourceLeafNode.xlsx,   
   SourceNotLeafNode.xlsx, TargetSurfaceTopographySourceNotLeafNode.xlsx,     
   TargetNotSurfaceTopographyOrSourceLeafNode.xlsx)   
respectively without the index column in mapdir  
 


In [72]:
import pandas as pd
from owlready2 import *

# Initialize directories
topdir = '/Users/tu/workspace/harmonization'
mapdir = f"{topdir}/maps/anatomy"
ontdir = f"{topdir}/ontology/renamedIRIs"

# Function that checks if IRI is a leaf node
def isLeafNode(iri):
    return len(iri.descendants()) == 1

# Load the ontology
onto = get_ontology(f"{ontdir}/whofic-renamed.owl").load()

# Read the Excel mapping table
df = pd.read_excel(f"{mapdir}/ICF-ICD-anatomy-v1.xlsx", header=0)

# Define new DataFrames
DF1 = pd.DataFrame()
DF2 = pd.DataFrame()
DF3 = pd.DataFrame()
DF4 = pd.DataFrame()
DF5 = pd.DataFrame()
DF6 = pd.DataFrame()

# Define SurfaceTopIRI
SurfaceTopIRI = onto.search_one(iri='http://id.who.int/icd/entity/687250607')

# Iterate through each row of the DataFrame
for _, row in df.iterrows():
    subjectIRI = onto.search_one(iri='http://id.who.int/icf/entity/' + str(row['subject_id']))
    objectIRI = onto.search_one(iri='http://id.who.int/icd/entity/' + str(row['object_id']))

    # Check conditions and add row to corresponding dataframe
    if SurfaceTopIRI in objectIRI.ancestors(): # If objectIRI is a descendant of SurfaceTopIRI
        DF1 = pd.concat([DF1, df.loc[_:_]])
    else:
        DF2 = pd.concat([DF2, df.loc[_:_]])

    if isLeafNode(subjectIRI): # If subjectIRI is a leafnode
        DF3 = pd.concat([DF3, df.loc[_:_]])
    else:
        DF4 = pd.concat([DF4, df.loc[_:_]])

    if SurfaceTopIRI in objectIRI.ancestors() and not isLeafNode(subjectIRI): # If objectIRI is a descendant of SurfaceTopIRI and subjectIRI is a leafnode
        DF5 = pd.concat([DF5, df.loc[_:_]])
    else:
        DF6 = pd.concat([DF6, df.loc[_:_]])

# Write Dataframes to Excel files
DF1.to_excel(f"{mapdir}/TargetSurfaceTopography.xlsx", index=False)
DF2.to_excel(f"{mapdir}/TargetNotSurfaceTopography.xlsx", index=False)
DF3.to_excel(f"{mapdir}/SourceLeafNode.xlsx", index=False)
DF4.to_excel(f"{mapdir}/SourceNotLeafNode.xlsx", index=False)
DF5.to_excel(f"{mapdir}/TargetSurfaceTopographySourceNotLeafNode.xlsx", index=False)
DF6.to_excel(f"{mapdir}/TargetNotSurfaceTopographyOrSourceLeafNode.xlsx", index=False)

## Filter ICHI maps into groups for experiments as bridging axioms

In [72]:
import pandas as pd
from owlready2 import *

# Initialize directories
topdir = '/Users/tu/workspace/harmonization'
mapdir = f"{topdir}/maps/anatomy"
ontdir = f"{topdir}/ontology/renamedIRIs"
classification = "ICHI"

# Function that checks if IRI is a leaf node
def isLeafNode(iri):
    return len(iri.descendants()) == 1

# Load the ontology
onto = get_ontology(f"{ontdir}/whofic-renamed.owl").load()

# Read the Excel mapping table
df = pd.read_excel(f"{mapdir}/ICHI-ICD-anatomy-v3.xlsx", header=0)

# Define new DataFrames
DF1 = pd.DataFrame()
DF2 = pd.DataFrame()
DF3 = pd.DataFrame()
DF4 = pd.DataFrame()
DF5 = pd.DataFrame()
DF6 = pd.DataFrame()

# Define SurfaceTopIRI
SurfaceTopIRI = onto.search_one(iri='http://id.who.int/icd/entity/687250607')

# Iterate through each row of the DataFrame
for _, row in df.iterrows():
    subjectIRI = onto.search_one(iri='http://id.who.int/ichi/entity/' + str(row['source']))
    objectIRI = onto.search_one(iri='http://id.who.int/icd/entity/' + str(row['target']))

    # Check conditions and add row to corresponding dataframe
    if SurfaceTopIRI in objectIRI.ancestors(): # If objectIRI is a descendant of SurfaceTopIRI
        DF1 = pd.concat([DF1, df.loc[_:_]])
    else:
        DF2 = pd.concat([DF2, df.loc[_:_]])

    if isLeafNode(subjectIRI): # If subjectIRI is a leafnode
        DF3 = pd.concat([DF3, df.loc[_:_]])
    else:
        DF4 = pd.concat([DF4, df.loc[_:_]])

    if SurfaceTopIRI in objectIRI.ancestors() and not isLeafNode(subjectIRI): # If objectIRI is a descendant of SurfaceTopIRI and subjectIRI is a leafnode
        DF5 = pd.concat([DF5, df.loc[_:_]])
    else:
        DF6 = pd.concat([DF6, df.loc[_:_]])

# Write Dataframes to Excel files
DF1.to_excel(f"{mapdir}/ICHITargetSurfaceTopography.xlsx", index=False)
DF2.to_excel(f"{mapdir}/ICHITargetNotSurfaceTopography.xlsx", index=False)
DF3.to_excel(f"{mapdir}/ICHISourceLeafNode.xlsx", index=False)
DF4.to_excel(f"{mapdir}/ICHISourceNotLeafNode.xlsx", index=False)
DF5.to_excel(f"{mapdir}/ICHITargetSurfaceTopographySourceNotLeafNode.xlsx", index=False)
DF6.to_excel(f"{mapdir}/ICHITargetNotSurfaceTopographyOrSourceLeafNode.xlsx", index=False)

## Add ICF-body-structure-to-ICD-anatomy maps as bridging axioms (if ICF sources are leaf nodes)
As a programmer, write the following python code without executing it. using the owlready2 API, read an OWL file from ontology/whofic-anatomy-harmonization.owl, and the mapping table map/anatomy/ICF-ICD-anatomy-v1 Excel spreadsheet. Use the first row as the header row. For each row after the header row, use the owlread2 API add OWL axiom to the loaded ontology as follows:
1. subjectIRI = 'http://id.who.int/icf/entity/' + (subject_id as a string) of the row
2. objectIRI = 'http://id.who.int/icd/entity/' + (object_id as a string) of the row
3. If subjectIRI is a leaf node, then
3.1 . If predicate_id is 'skos:BroadMatch' then add the axiom objectIRI isSubClassOf subjectIRI
3.2. If predicate_id is 'skos:NarrowMatch' then add the axiom subjectIRI isSubClassOf objectIRI
3.3 . If the predicate_id is 'skos:ExactMatch', then add the axiom subjectIRI isEquivalentTo objectIRI
endif
Save the ontology to the file whofic-anatomy-harmonization-1.owl

In [8]:
topdir = '/Users/tu/workspace/harmonization'
mapdir = F"{topdir}/maps/anatomy"
ontdir = F"{topdir}/ontology/renamedIRIs"

import pandas as pd
from owlready2 import *

# Load the ontology
ontology = get_ontology(F"{ontdir}/whofic-renamed.owl").load()

# Load the Excel mapping file
df = pd.read_excel(F"{mapdir}/ICF-ICD-anatomy-v1.xlsx")

# Iterate rows in DataFrame
count = 0
for idx, row in df.iterrows():
    subject_iri = IRIS['http://id.who.int/icf/entity/' + str(row['subject_id'])]
    object_iri = IRIS['http://id.who.int/icd/entity/' + str(row['object_id'])]

    if not subject_iri.descendants():
        print(F"{subject_iri} {row['predicate_id']} {object_iri}")
        count = count + 1
    # Create corresponding axioms based on predicate_id
        if row['predicate_id'] == 'skos:BroadMatch':
            axiom = object_iri.is_a.append(subject_iri) 
        elif row['predicate_id'] == 'skos:NarrowMatch':
            axiom = subject_iri.is_a.append(object_iri) 
        elif row['predicate_id'] == 'skos:ExactMatch':
            axiom = subject_iri.equivalent_to.append(object_iri)
print("Number of axioms: "+ count)
# Save the ontology with the new axioms
ontology.save(F"{ontdir}/whofic-anatomy-icfleaves-harmonization.owl")


## Add ICF-body-structure-to-ICD-anatomy maps as bridging axioms (if ICD targets are not part of Surface topography)
As a programmer, write the following python code without executing it. using the owlready2 API, read an OWL file from ontology/whofic-anatomy-harmonization.owl, and the mapping table map/anatomy/ICF-ICD-anatomy-v1 Excel spreadsheet. Use the first row as the header row. For each row after the header row, use the owlread2 API add OWL axiom to the loaded ontology as follows:
1. subjectIRI = 'http://id.who.int/icf/entity/' + (subject_id as a string) of the row
2. objectIRI = 'http://id.who.int/icd/entity/' + (object_id as a string) of the row
3. If objectIRI is not a descendant of http://id.who.int/icd/entity/687250607, then
3.1 . If predicate_id is 'skos:BroadMatch' then add the axiom objectIRI isSubClassOf subjectIRI
3.2. If predicate_id is 'skos:NarrowMatch' then add the axiom subjectIRI isSubClassOf objectIRI
3.3 . If the predicate_id is 'skos:ExactMatch', then add the axiom subjectIRI isEquivalentTo objectIRI
endif
Save the ontology to the file whofic-anatomy-nosurfacet-harmonization.owl

In [ ]:
import pandas as pd
from owlready2 import *

topdir = '/Users/tu/workspace/harmonization'
mapdir = F"{topdir}/maps/anatomy"
ontdir = F"{topdir}/ontology/renamedIRIs"

# Load the ontology
ontology = get_ontology(F"{ontdir}/whofic-renamed.owl").load()

# Load the Excel mapping file
df = pd.read_excel(F"{mapdir}/ICF-ICD-anatomy-v1.xlsx")

# Reference IRI for comparison
compareIRI = onto['http://id.who.int/icd/entity/687250607'] 

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    subject_iri = IRIS['http://id.who.int/icf/entity/' + str(row['subject_id'])]
    object_iri = IRIS['http://id.who.int/icd/entity/' + str(row['object_id'])]
    predicate_id = str(row['predicate_id'])

    # Construct the IRI
    subjectIRI = onto['http://id.who.int/icf/entity/' + subject_id]
    objectIRI = onto['http://id.who.int/icd/entity/' + object_id]

    # Check if objectIRI is not a descendant of compareIRI:
    if compareIRI not in objectIRI.ancestors(): 

        # Add axioms based on predicate_id 
        if predicate_id == 'skos:BroadMatch':
            objectIRI.is_a.append(subjectIRI)
        elif predicate_id == 'skos:NarrowMatch':
            subjectIRI.is_a.append(objectIRI)
        elif predicate_id == 'skos:ExactMatch':
            EquivalentTo([subjectIRI, objectIRI])

# Save the updated ontology
onto.save("whofic-anatomy-nosurfacet-harmonization.owl")



## Add ICF-body-structure-to-ICD-anatomy maps as bridging axioms (if ICD targets are not part of Surface topography or if the ICF source is a leaf node)
As a programmer, write the following python code without executing it. 
topdir = '/Users/tu/workspace/harmonization' 
mapdir = F"{topdir}/maps/anatomy" 
ontdir = F"{topdir}/ontology/renamedIRIs" 

using the owlready2 API, read an OWL file from F"{ontdir}whofic-renamed.owl", 
Read the mapping table F"{mapdir}TargetNotSurfaceTopographyOrSourceLeafNode" Excel spreadsheet. 

Use the first row as the header row. 
  subjectIRI = onto.search_one(iri='http://id.who.int/icf/entity/' + (subject_id as a string)) of the row 
  objectIRI = onto.search_one(iri='http://id.who.int/icd/entity/' + (object_id as a string)) of the row 
  
  For each row after the header row, use the owlread2 API to add OWL axiom to the loaded ontology as follows:
  If predicate_id is 'skos:BroadMatch' then add the axiom objectIRI isSubClassOf subjectIRI
  If predicate_id is 'skos:NarrowMatch' then add the axiom subjectIRI isSubClassOf objectIRI
  If the predicate_id is 'skos:ExactMatch', then add the axiom subjectIRI isEquivalentTo objectIRI

Save the ontology to the file whofic-anatomy-nosurftop-or-leaf-harmonization.owl

In [78]:

import pandas as pd
from owlready2 import *

# Define directories
topdir = '/Users/tu/workspace/harmonization' 
mapdir = f"{topdir}/maps/anatomy" 
ontdir = f"{topdir}/ontology/renamedIRIs" 

# Load the ontology
onto = get_ontology(f"{ontdir}/whofic-renamed.owl").load()

# Load the Excel mapping table
df = pd.read_excel(f"{mapdir}/TargetNotSurfaceTopographyOrSourceLeafNode.xlsx", header=0)
count = 0
# Iterate over each row in the file (skipping the header row)
for _, row in df.iterrows():
    # Create the IRI for each subject and object
    subjectIRI = onto.search_one(iri='http://id.who.int/icf/entity/' + str(row['subject_id']))
    objectIRI = onto.search_one(iri='http://id.who.int/icd/entity/' + str(row['object_id']))

    # Add the axioms to the ontology based on the predicate_id
    predicate_id = row['predicate_id']
    if predicate_id == 'skos:BroadMatch':
        objectIRI.is_a.append(subjectIRI)
    elif predicate_id == 'skos:NarrowMatch':
        subjectIRI.is_a.append(objectIRI)
    elif predicate_id == 'skos:ExactMatch':
        subjectIRI.equivalent_to.append(objectIRI)

# Save the modified ontology
onto.save(F"{ontdir}/whofic-anatomy-nosurftop-or-leaf-harmonization.owl")


## Add ICHI anatomy-target-to-ICD-anatomy maps as bridging axioms


Check to see if there is any cycles in the ICHI-ICD mappings
Write the python code to do the following:
- topdir = '/Users/tu/workspace/harmonization'
- mapdir = F"{topdir}/maps/anatomy"
- read the excel file F"{mapdir}/ICHI-ICD-anatomy-v1.xlsx"
- Use the first row as the header row. 
- For the following rows, check if there are two rows where source and target are the same but predicate_id are different
- print out such pairs of rows


In [9]:
import pandas as pd

# Directories
topdir = '/Users/tu/workspace/harmonization'
mapdir = f"{topdir}/maps/anatomy"

# Read the excel file
df = pd.read_excel(f"{mapdir}/ICHI-ICD-anatomy-v1.xlsx")

# Group the DataFrame by 'subject_id' and 'object_id'
grouped = df.groupby(['source', 'target'])

# Find groups where 'predicate_id' has more than one unique value
for name, group in grouped:
    if group['predicate_id'].nunique() > 1:
        print(group)


In [ ]:
topdir = '/Users/tu/workspace/harmonization'
mapdir = F"{topdir}/maps/anatomy"
ontdir = F"{topdir}/ontology/renamedIRIs"

import pandas as pd
from owlready2 import *

# Load the ontology
ontology = get_ontology(F"{ontdir}/whofic-renamed.owl").load()

# Load the Excel mapping file
df = pd.read_excel(F"{mapdir}/ICHI-ICD-anatomy-v1.xlsx")

# Iterate rows in DataFrame
for idx, row in df.iterrows():
    subject_iri = IRIS[row['source']]
    object_iri = IRIS[row['target']]


    # Create corresponding axioms based on predicate_id
    try: 
        if row['predicate_id'] == 'br':
            axiom = object_iri.is_a.append(subject_iri) 
        elif row['predicate_id'] == 'eq':
            axiom = subject_iri.equivalent_to.append(object_iri)
        elif row['predicate_id'] == 'na':
            axiom = subject_iri.is_a.append(object_iri)
    except:
        print(f"source {row['source']}; target {row['target']}; predicate {row['predicate_id']}")

# Save the ontology with the new axioms
ontology.save(F"{ontdir}/whofic-ichi-icd-anatomy-harmonization.owl")


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
source http://id.who.int/ichi/entity/1154085341; target http://id.who.int/icd/entity/947150833; predicate na
source http://id.who.int/ichi/entity/1154085341; target http://id.who.int/icd/entity/40907641; predicate na


In [15]:
subject_iri = IRIS['http://id.who.int/ichi/entity/1652076574']
object_iri = IRIS[ 'http://id.who.int/icd/entity/1885434231']
for c in object_iri.subclasses():
    print(c)
subject_iri.isa.append(object_id)

entity.1093096992
entity.1204996303
entity.1213923926
entity.2034447649
entity.1308284441
entity.1358605327
entity.1366916688
entity.1385726880
entity.1474796775
entity.1601398422
entity.1934266287
entity.1656025598
entity.1712049110
entity.1799284202
entity.1855926695
entity.1898163990
entity.1973196094
entity.2042058087
entity.207306511
entity.2081849642
entity.351965526
entity.460544975
entity.46930929
entity.604514306
entity.622249575
entity.842965671
entity.891523663


## Change the IRIs of ICF and ICHI entities so that they are distinguuishable
Use the owlready2 API, read an OWL file whofic-2024-01-21.owl
find all descendants of 'http://id.who.int/icd/entity/1405434703' and change their names from 'http://id.who.int/icd/entity/nnnnnnnnn' to 'http://id.who.int/icf/entity/nnnnnnnnn'. save the ontology as whofic-renamed.owl. Ditto for ICHI entities (except anatomy and topography, which ICHI shares with ICD)

In [ ]:
from owlready2 import *

dir = "/Users/tu/workspace/evalicdmappings/ontologies/"
# Load your ontology
onto = get_ontology(f"file://{dir}whofic-2024-01-21.owl").load()

# top-level ICF 
class_to_change = onto.search_one(iri="http://id.who.int/icd/entity/1405434703")
if class_to_change:
    # Change the name of the class itself
    class_to_change.iri = class_to_change.iri.replace('/icd/', '/ichi/')

    # Fetch subclasses and change IRIs
    for sub_class in class_to_change.descendants():
        old_iri = sub_class.iri
        new_iri = old_iri.replace("/icd/", "/icf/")
        sub_class.iri = new_iri

# top-level ICHI minus anatomy & topography and topology
classes_to_rename = [
    "http://id.who.int/icd/entity/1190112446",
    "http://id.who.int/icd/entity/1399457851",
    "http://id.who.int/icd/entity/1338961517",
    "http://id.who.int/icd/entity/337502477",
    "http://id.who.int/icd/entity/974232433",
    "http://id.who.int/icd/entity/448900003",
    "http://id.who.int/icd/entity/89604631"
]

for class_iri in classes_to_rename:
    class_to_change = onto.search_one(iri=class_iri)
    
    if class_to_change:
        # Change the name of the class itself
        class_to_change.iri = class_to_change.iri.replace('/icd/', '/ichi/')

        # Change the names of all its descendants (subclasses)
        for descendant in class_to_change.descendants():
            descendant.iri = descendant.iri.replace('/icd/', '/ichi/')

# Save the modified ontology
onto.save(f"{dir}whofic-renamed.owl", format="rdfxml")


## Add rdfs:label to ICD transport tree, using skos:prefLabel

In [ ]:
from owlready2 import *

#inputFile = "/Users/tu/workspace/harmonization/ontology/CPC/icd-transport-equipment.owl"
inputFile = "/Users/tu/Documents/Dropbox/Ontologies/ICD11.owl"
#inputFile = "/Users/tu/Documents/Dropbox/Ontologies/ICD10.owl"
outputFile = "/Users/tu/Documents/Dropbox/Ontologies/ICD11-label.owl"

In [ ]:
onto=get_ontology(inputFile).load()
classes = onto.classes()
for c in classes:
    c.label = c.prefLabel
onto.save(file = outputFile, format="rdfxml")

## Extracting anatomical/functional targets

In [5]:
%cd '/Users/tu/Documents/Dropbox/WHO (1)/ICHI/Versions/ICHI - IRI'  
!robot extract --prefix 'ichi: http://id.who.int/ichi/entity/' --method MIREOT --input ICHI-IRI-Modified.owl --branch-from-term ichi:229313065 --output ICHI-Anatomy-Targets.owl


/Users/tu/Documents/Dropbox/WHO (1)/ICHI/Versions
zsh:1: command not found: robot


In [11]:
import pandas as pd
import re

# Read the two files
def read_file(file_path):
    try:
        # First try to read as tab-separated file
        df = pd.read_csv(file_path, sep='\t', header=None, dtype=str, encoding='utf-8')
        return df
    except UnicodeDecodeError:
        try:
            # Try with latin-1 encoding
            df = pd.read_csv(file_path, sep='\t', header=None, dtype=str, encoding='latin1')
            return df
        except:
            pass
    
    # If tab-separated reading fails, try reading as plain text
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = [line.strip() for line in f.readlines() if line.strip()]
        return pd.DataFrame(lines, columns=['entry'])
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin1') as f:
            lines = [line.strip() for line in f.readlines() if line.strip()]
        return pd.DataFrame(lines, columns=['entry'])

# Function to extract code and text from an entry
def extract_code_and_text(entry):
    # Attempt to split the entry by whitespace
    # Assumes the code comes first, followed by text
    parts = entry.strip().split(maxsplit=1)
    if len(parts) >= 2:
        return parts[0], parts[1]
    else:
        return entry.strip(), ""  # Return just the code if no text is found

# File paths
f1_path = "/Users/tu/Documents/Dropbox/WHO (1)/HIRG/JQNMHCShortList2023.txt"
f2_path = "/Users/tu/Documents/Dropbox/WHO (1)/HIRG/JQNMHCShortList2018.txt"

# Read files
df1 = read_file(f1_path)  # 2023 file
df2 = read_file(f2_path)  # 2018 file

# Create dictionaries to store entries with their code and text
entries_2023 = {}
entries_2018 = {}

# Process 2023 entries
if 'entry' in df1.columns:
    # If we read it as plain text with 'entry' column
    for entry in df1['entry']:
        code, text = extract_code_and_text(entry)
        entries_2023[entry.strip()] = (code, text)
else:
    # If we read it as tab-separated
    for idx, row in df1.iterrows():
        if len(row) >= 2:  # If there are at least 2 columns
            code = row[0]
            text = row[1]
        else:
            # If there's only one column, try to split it
            entry = row[0]
            code, text = extract_code_and_text(entry)
        entries_2023[f"{code} {text}".strip()] = (code, text)

# Process 2018 entries
if 'entry' in df2.columns:
    # If we read it as plain text with 'entry' column
    for entry in df2['entry']:
        code, text = extract_code_and_text(entry)
        entries_2018[entry.strip()] = (code, text)
else:
    # If we read it as tab-separated
    for idx, row in df2.iterrows():
        if len(row) >= 2:  # If there are at least 2 columns
            code = row[0]
            text = row[1]
        else:
            # If there's only one column, try to split it
            entry = row[0]
            code, text = extract_code_and_text(entry)
        entries_2018[f"{code} {text}".strip()] = (code, text)

# Find differences
added_entries = set(entries_2023.keys()) - set(entries_2018.keys())
deleted_entries = set(entries_2018.keys()) - set(entries_2023.keys())

# Create lists for added and deleted entries
added_items = [
    {"Code": entries_2023[entry][0], "Text": entries_2023[entry][1]} 
    for entry in sorted(added_entries)
]

deleted_items = [
    {"Code": entries_2018[entry][0], "Text": entries_2018[entry][1]} 
    for entry in sorted(deleted_entries)
]

# Create DataFrames for added and deleted entries
added_df = pd.DataFrame(added_items)
deleted_df = pd.DataFrame(deleted_items)

# Save as tab-separated files
if not added_df.empty:
    added_df.to_csv('added_entries.tsv', sep='\t', index=False, header=True)
else:
    with open('added_entries.tsv', 'w') as f:
        f.write("Code\tText\n")

if not deleted_df.empty:
    deleted_df.to_csv('deleted_entries.tsv', sep='\t', index=False, header=True)
else:
    with open('deleted_entries.tsv', 'w') as f:
        f.write("Code\tText\n")

# Print results as tab-separated text
print("ADDED ENTRIES (in 2023 but not in 2018):")
print("Code\tText")
if added_items:
    for item in added_items:
        print(f"{item['Code']}\t{item['Text']}")
else:
    print("No added entries found.")

print("\nDELETED ENTRIES (in 2018 but not in 2023):")
print("Code\tText")
if deleted_items:
    for item in deleted_items:
        print(f"{item['Code']}\t{item['Text']}")
else:
    print("No deleted entries found.")

# Also save as tab-separated text files
with open('added_entries.txt', 'w', encoding='utf-8') as f:
    f.write("ADDED ENTRIES (in 2023 but not in 2018):\n")
    f.write("Code\tText\n")
    if added_items:
        for item in added_items:
            f.write(f"{item['Code']}\t{item['Text']}\n")
    else:
        f.write("No added entries found.")

with open('deleted_entries.txt', 'w', encoding='utf-8') as f:
    f.write("DELETED ENTRIES (in 2018 but not in 2023):\n")
    f.write("Code\tText\n")
    if deleted_items:
        for item in deleted_items:
            f.write(f"{item['Code']}\t{item['Text']}\n")
    else:
        f.write("No deleted entries found.")

print(f"\nTotal added entries: {len(added_entries)}")
print(f"Total deleted entries: {len(deleted_entries)}")
print("\nResults saved to:")
print("- added_entries.tsv and added_entries.txt")
print("- deleted_entries.tsv and deleted_entries.txt")

ADDED ENTRIES (in 2023 but not in 2018):
Code	Text
00.66	Percutaneous transluminal coronary angioplasty [PTCA] or coronary
00.70	Revision of hip replacement, both acetabular and femoral components
00.71	Revision of hip replacement, acetabular component
00.72	Revision of hip replacement, femoral component
00.73	Revision of hip replacement, acetabular liner and/or femoral head only
00.74	Hip replacement bearing surface, metal on polyethylene
00.75	Hip replacement bearing surface, metal-on-metal
00.76	Hip replacement bearing surface, ceramic-on-ceramic
00.77	Hip replacement bearing surface, ceramic-on-polyethylene
13.1	Intracapsular extraction of lens
13.2	Extracapsular extraction of lens by linear extraction technique
13.3	Extracapsular extraction of lens by simple aspiration (and irrigation) technique
13.4	Extracapsular extraction of lens by fragmentation and aspiration technique
13.5	Other extracapsular extraction of lens
13.6	Other cataract extraction
13.7	Insertion of prosthetic lens

Set directory D = '/Users/tu/workspace/evalmappings/bin/CodeFusion_osx-x64_0.9.5/
Use panda to read the file:
f = f"{D}/ICHI-NCSP_plus 2024-09-30 maps-checked.xlsx"
Create an output Excel file mismatches.xlsx in directory D
for each entry in f add the entry to the output file
If the value of column 'ICHI code' is 'NoMatch' and the value of column 'MatchLevel' is not 'NoMatch'
or If the value of column 'ICHI code' is not 'NoMatch' and the value of column 'ICHI code' does not match the value in column code

In [23]:

import pandas as pd

# Set the directory
D = '/Users/tu/workspace/evalmappings/bin/CodeFusion_osx-x64_0.9.5/'

# Read the Excel file
file_path = f"{D}/ICHI-NCSP_plus 2024-09-30 maps-checked.xlsx"
df = pd.read_excel(file_path)

# Create a mask for the mismatches based on the conditions provided
mismatch_condition = (
    (df['ICHI code'] == 'NoMatch') & (df['MatchLevel'] != 'NoMatch') |
    (df['ICHI code'] != 'NoMatch') & (df['ICHI code'] != df['code'])
)

# Filter the DataFrame based on the mismatch condition
mismatches = df[mismatch_condition]

# Create the output Excel file
output_file_path = f"{D}/mismatches.xlsx"
mismatches.to_excel(output_file_path, index=False)

print(f"Mismatches have been written to {output_file_path}")


Mismatches have been written to /Users/tu/workspace/evalmappings/bin/CodeFusion_osx-x64_0.9.5//mismatches.xlsx
